<a href="https://colab.research.google.com/github/codeREXus/langchain-learnings/blob/main/Langchain1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install  langchain-anthropic

In [ ]:
from langchain_core.runnables import RunnableParallel
from langchain_anthropic import ChatAnthropic
from langchain.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda


from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [2]:
from google.colab import userdata


In [ ]:
from google.colab import userdata
llm = ChatAnthropic(
    model="claude-3-opus-20240229",  # or claude-3-sonnet-20240229, claude-3-haiku-20240307
    temperature=0.5,
    api_key=userdata.get('anthropic-api-key')
)
chain= {
    "summary":ChatPromptTemplate.from_template("Summarize this text: {text}") | llm ,
    "translation":ChatPromptTemplate.from_template("Translate this text from English to French: {text}") | llm,
    "sentiment":ChatPromptTemplate.from_template("Classify the sentiment of the following text: {text}") | llm
}

In [ ]:
result = chain['sentiment'].invoke({"text":"I really  enjoyed the conference today"})
print(result.content)

The sentiment of the text "I really enjoyed the conference today" is positive. The word "enjoyed" expresses a positive feeling or experience, and the adverb "really" intensifies the positive sentiment, indicating that the speaker had a very pleasant and satisfying time at the conference.


In [ ]:

# Define a function to ensure proper formatting
def format_prompt(variables):
    return prompt.format(**variables)

In [ ]:
description = """
    Retrieve the names and email addresses of all customers from the 'customers' table who have made a purchase in the last 30 days.
    The table 'purchases' contains a column 'purchase_date'
"""

template = """
    Generate an SQL query based on the {description}

    SQL Query:

"""
prompt = ChatPromptTemplate.from_template(template)

# Create the LCEL chain
sql_generation_chain = (
    RunnableLambda(format_prompt)
    | llm
)

# Run the chain
sql_query = sql_generation_chain.invoke({"description": description})
print(sql_query.content)

Lets do a task

In [ ]:
from langchain.chains import LLMChain, SequentialChain
template=""" Your job is to get a famous dish from the location that user suggests. {location}
              your response:"""
prompt_temp = PromptTemplate(template= template, input_variables=['location'])
location_chain=LLMChain(llm=llm , prompt=prompt_temp , output_key= "meal")


template="""Given a meal {meal}, give a short and simple recepie on how to make that dish"""
prompt_temp = PromptTemplate(template=template, input_variables=["meal"])
dish_chain =LLMChain(llm= llm , prompt=prompt_temp, output_key="recepie")

template= """GIven a recepie{recepie}, estimate what time would a begginer take to make it"""
prompt_temp = PromptTemplate(template = template,input_variables=["recepie"])
time_chain=LLMChain(llm= llm, prompt=prompt_temp,output_key="time")

In [ ]:
chain = SequentialChain(chains=[location_chain, dish_chain, time_chain],
                         input_variables=['location'],
                         output_variables=['meal','recepie','time'],
                        verbose=True
                        )
chain.invoke(input={"location":"India"})